# Business Problem

A client - an independent movie company that prefers to remain anonymous - is interested in entering the streaming space.  The client recognizes that this space is competitive due to present offering.  However, the client still believes there is an opportunity based on its marketing analysis and backlog of independent films.

Before building the streaming service, the client has requested KBO Analytics to create a recommendation system.  KBO Analytics will address the first phase of this project by building a proof-of-concept based on the MovieLens dataset.

# Data Understanding

The data for examing the aforementioned problem comes from the following source: [MovieLens](https://grouplens.org/datasets/movielens/latest/)

Before beginning to create a recommendation system, I want to examine and become familiar with the dataset. I will conduct exploratory data analysis in order to understand the dataset attributes, which includes, but not limited to the following:

1. Number of Columns
2. Number of Rows
3. Column Names
4. Format of the data in each column

In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

# Data Preparation

# Modeling

# Overall Conclusion and Recommendations

## Overall Conclusion

## Recommendations